In [1]:
%matplotlib inline
from matplotlib import pyplot as plt
from matplotlib import dates as mdate
from datetime import datetime
import json, re, pymysql,  pandas
from pandas.io.json import json_normalize
#pandas.options.display.float_format = "{:.0f}".format
pandas.set_option('display.max_rows', None)
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
import ipywidgets as widgets 
from IPython.display import display, clear_output, Markdown

In [2]:
class BabiloniaPriceDataAnalytics:
    __SOURCES=['CEAGESP'] # coming soon ... 'STMARCHE'
    __CEAGESP=None #do not need to query db everytime
    __CEAGESP_refData=None
    __CEAGESP_priceLevel=['Menor','Comun','Maior']
    def __init__(self, config_path):
        with open(config_path) as json_data_file:
            data = json.load(json_data_file)
            url = data['SQLALCHEMY_DATABASE_URI']
            self.__MUX_PORT_THRESHOLD = data['MUX_PORT_THRESHOLD']
        user, password, host, database = re.match('mysql://(.*?):(.*?)@(.*?)/(.*)', url).groups()
        self.__CONN = pymysql.connect(host=host,port=3306,user=user,passwd=password,db=database)
        with open("./ceagesp.json") as ceagesp:
            self.__CEAGESP_refData = json.load(ceagesp)
    def get_sources(self):
        return self.__SOURCES
    def get_ceagesp_refdata(self):
        return self.__CEAGESP_refData
    def get_ceagesp_price_level(self):
        return self.__CEAGESP_priceLevel
    def get_ceagesp_data(self):
        if self.__CEAGESP is None:
            query="""
                SELECT DATE, DATA
                FROM farmland.PRICES 
                WHERE SOURCE='CEAGESP';
                """
            df = pandas.read_sql_query(query, self.__CONN)
            pandas.to_datetime(df['DATE'])
            df.set_index('DATE', inplace=True)
            self.__CEAGESP = df
            
            if len(self.__CEAGESP) == 0:
                print ("get_ceagesp_data: no data found for!!!")
                return None
            else:
                print("get_ceagesp_data: data found. Memory usage:",
                      int(self.__CEAGESP.memory_usage(deep=True).sum()/1024),"KB")            
        return self.__CEAGESP
    
    def get_ceagesp_dates(self):
        self.get_ceagesp_data()
        return self.__CEAGESP.index

    def get_product_price(self, product, product_type, price_type):
        self.get_ceagesp_data()
        results =[]
        for index, row in self.__CEAGESP.iterrows():
            jd = json_normalize(json.loads(json.loads(row['DATA'])))
            price = float( jd[(jd['Produto'] == product)& (jd['Classificacao']==product_type)][price_type].values[0].replace(',', '.'))
            result={'DATE': index, 'PRICE': price}
            results.append(result)
        df = pandas.DataFrame(results) 
        df.set_index('DATE', inplace=True)
        return df
    """
    Using this function, you will explore Babilonia prices data
    """
    def plot_price(self):
        outt = widgets.Output()
        dropdown_src = widgets.Dropdown(options=self.get_sources(), description='Source:')
        btn_search = widgets.Button(description='Retrieve data',button_style='warning')
        def btn_search_clicked(b):
            with outt:
                clear_output()
                if self.get_ceagesp_data() is not None:
                    dropdown_product.disabled = False           
                else:
                    dropdown_product.disabled = True
                    dropdown_class.disabled = True
                    dropdown_priceLevel.disabled = True

        btn_search.on_click(btn_search_clicked)
        menu = widgets.HBox([dropdown_src, btn_search])
        def dropdown_product_on_event(change):
            if change['type'] == 'change' and change['name'] == 'value':
                dropdown_class.disabled = False
                dropdown_priceLevel.disabled = False
                dropdown_class.options = list(self.get_ceagesp_refdata()[change['new']].keys())

        dropdown_product = widgets.Dropdown(options=list(self.get_ceagesp_refdata().keys()), 
                                             description='Product:', disabled=True)
        dropdown_product.observe(dropdown_product_on_event)
        
        dropdown_class = widgets.Dropdown(options=[''], 
                                             description='Class:', disabled=True)
        
        dropdown_priceLevel = widgets.Dropdown(options=self.get_ceagesp_price_level(), 
                                             description='Price level:', disabled=True)
        
        btn_plot = widgets.Button(description='Plot!',button_style='danger')
        
        def on_btn_plot_clicked(b):
            with outt:
                clear_output()
                data = self.get_product_price(dropdown_product.value,
                                              dropdown_class.value,
                                              dropdown_priceLevel.value)
                info = self.get_ceagesp_refdata()[dropdown_product.value][dropdown_class.value]
                print("Product:",dropdown_product.value,
                      " Class:", dropdown_class.value,
                      " Price:", dropdown_priceLevel.value,
                      " Unit:", info['unit'],
                      " quilo:", info['quilo'])
                plt.rcParams['figure.figsize'] = [15,7]
                fig,ax = plt.subplots()
                plt.xticks(rotation=45)
                plt.plot(data, linestyle='solid', marker='None')
                plt.show()
        btn_plot.on_click(on_btn_plot_clicked)
        filters = widgets.HBox((dropdown_product,dropdown_class, dropdown_priceLevel, btn_plot))
        return widgets.VBox([menu, filters, outt])


In [3]:
bpda = BabiloniaPriceDataAnalytics('../../server/app/config.json')
#data = json.loads(json.loads(bpda.get_ceagesp_data().iloc[0]['DATA']))# first df element

In [4]:
bpda.plot_price()